In [41]:
from collect_data import *

In [42]:
megsp_path = os.path.join(meg_path, 'collect_data/megsp')
audio_path = os.path.join(meg_path, 'collect_data/audio')
megsp_list = os.listdir(megsp_path)
audio_list = os.listdir(audio_path)

In [43]:
audio_list

['audio___0_lw1_0.pt',
 'audio___0_lw1_1.pt',
 'audio___0_lw1_2.pt',
 'audio___0_lw1_3.pt',
 'audio___1_cable_spool_fort_0.pt',
 'audio___1_cable_spool_fort_1.pt',
 'audio___1_cable_spool_fort_2.pt',
 'audio___1_cable_spool_fort_3.pt',
 'audio___1_cable_spool_fort_4.pt',
 'audio___1_cable_spool_fort_5.pt',
 'audio___2_easy_money_0.pt',
 'audio___2_easy_money_1.pt',
 'audio___2_easy_money_2.pt',
 'audio___2_easy_money_3.pt',
 'audio___2_easy_money_4.pt',
 'audio___2_easy_money_5.pt',
 'audio___2_easy_money_6.pt',
 'audio___2_easy_money_7.pt']

In [44]:
audio_tensor_list = []

for file_name in audio_list:
    file_path = os.path.join(audio_path, file_name)
    tensor = torch.load(file_path)
    print('DIMENSION ' + file_name + ': ', tensor.shape)
    audio_tensor_list.append(tensor)

audio_tensor = torch.cat(audio_tensor_list, dim=0)
print('DIMENSION_AUDIO_TENSOR: ', audio_tensor.shape)

DIMENSION audio___0_lw1_0.pt:  torch.Size([180, 257, 376])
DIMENSION audio___0_lw1_1.pt:  torch.Size([139, 257, 376])
DIMENSION audio___0_lw1_2.pt:  torch.Size([225, 257, 376])
DIMENSION audio___0_lw1_3.pt:  torch.Size([124, 257, 376])
DIMENSION audio___1_cable_spool_fort_0.pt:  torch.Size([300, 257, 376])
DIMENSION audio___1_cable_spool_fort_1.pt:  torch.Size([325, 257, 376])
DIMENSION audio___1_cable_spool_fort_2.pt:  torch.Size([231, 257, 376])
DIMENSION audio___1_cable_spool_fort_3.pt:  torch.Size([249, 257, 376])
DIMENSION audio___1_cable_spool_fort_4.pt:  torch.Size([282, 257, 376])
DIMENSION audio___1_cable_spool_fort_5.pt:  torch.Size([116, 257, 376])
DIMENSION audio___2_easy_money_0.pt:  torch.Size([257, 257, 376])
DIMENSION audio___2_easy_money_1.pt:  torch.Size([190, 257, 376])
DIMENSION audio___2_easy_money_2.pt:  torch.Size([428, 257, 376])
DIMENSION audio___2_easy_money_3.pt:  torch.Size([378, 257, 376])
DIMENSION audio___2_easy_money_4.pt:  torch.Size([319, 257, 376])
DI